# Bosonic.py

In [1]:
import numpy as np
from scipy.special import binom 
from scipy.sparse import dok_matrix, linalg
from scipy import linalg as linalg_d
from joblib import Memory
import random
import plotly.graph_objects as go
from joblib import Parallel, delayed
from numba import jit, prange

location = './cachedir'
memory = Memory(location, verbose=0)

In [11]:
# Funciones auxiliares optimiadas
@jit
def int_to_tuple(n, b, digits = None):
    r = []
    re = n 
    while re != 0:
        r.append(re%b)
        re = re // b
    if digits is not None:
        if len(r)<digits:
            for i in range(0,digits-len(r)):
                r.append(0)
    r.reverse()
    return r

@jit
def _create_basis(m, d):
    base = []
    index = 0
    for x in range(0,(d+1)**max(d,m)):
        x = int_to_tuple(x,m+1,d)
        if sum(x) == m and len(x) == d:
            base.append(x)
            index += 1
    return base

class fixed_basis:

    # Convierte a un enterno n a su escritura en base b
    def _int_to_tuple(self, n, b, digits = None):
        rep = np.base_repr(n, b)
        rep_int = [int(x,b) for x in rep] 
        if digits is not None:
            zeros = [0 for i in range(0,digits-len(rep))]
            return zeros + rep_int
        else:
            return rep_int
    
    # Revierte la transformacion anterior
    def tuple_to_int(self, t):
        b = self.d-1
        l = len(t)
        s = [t[k]*b**(l-k-1) for k in range(0,l)]
        return sum(s)

    # Convierte el vector en su representacion
    def vect_to_repr(self, vect):
        for i, k in enumerate(vect):
            if k == 1. or k == 1:
                break
        return self.base[i,:]
            
    def rep_to_vect(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return self.canonicals[:,i]
        else:
            None
    
    def rep_to_index(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return i
        else:
            None

    @staticmethod
    def rep_to_exi(rep):
        r = []
        for i, k in enumerate(rep):
            r += [i for x in range(0,k)]
        return r

    # Crea base de M particulas en D estados (repr y base canonica)
    def create_basis(self, m, d):
        length = int(binom(d+m-1,d-1))
        base = np.array(_create_basis(m, d))
        # Asignamos a cada uno de ellos un canónico
        length = int(binom(d+m-1,d-1))
        x = [1.0 for j in range(0,length)]
        canonicals = np.diag(x)
        return base, canonicals
    
    def __init__(self, m, d):
        self.m = m
        self.d = d
        self.size = int(binom(d+m-1,d-1))
        (self.base, self.canonicals) = self.create_basis(m, d)

# Matrices de aniquilación y creación endomórficas. Estan fuera de la clase para poder ser cacheadas        
@memory.cache    
def bdb(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    if i != j:
        for k, v in enumerate(basis.base):
            if v[j] != 0:
                dest = list(v.copy())
                dest[j] -= 1
                dest[i] += 1
                tar = basis.rep_to_index(dest)
                mat[tar, k] = np.sqrt(v[i]+1)*np.sqrt(v[j])
    else:
        for k, v in enumerate(basis.base):
            if v[j] != 0:
                mat[k, k] = v[i] 
    return mat

@memory.cache    
def bbd(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    if i != j:
        for k, v in enumerate(basis.base):
            if v[i] != 0:
                dest = list(v.copy())
                dest[i] -= 1
                dest[j] += 1
                tar = basis.rep_to_index(dest)
                mat[tar, k] = np.sqrt(v[j]+1)*np.sqrt(v[i])
    else:
        for k, v in enumerate(basis.base):
            mat[k, k] = v[i]+1
    return mat

# Matrices de aniquilación y creación.Toman la base de origen y destino (basis_o, basis_d) resp
@memory.cache   
def b(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        if v[i] != 0:
            dest = list(v.copy())
            dest[i] -= 1   
            tar = basis_d.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[i])
    return mat


@memory.cache   
def bd(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        dest = list(v.copy())
        dest[i] += 1   
        tar = basis_d.rep_to_index(dest)
        mat[tar, k] = np.sqrt(v[i]+1)
    return mat

# Devuelve la matriz gamma asociada a la descomposición (M,N-M) del vector
def gamma(basis, m, vect, m_basis = None, nm_basis = None):
    d = basis.d
    if not m_basis or not nm_basis:
        m_basis = fixed_basis(m, d)
        nm_basis = fixed_basis(basis.m-m,d)
    mat = dok_matrix((m_basis.size, nm_basis.size), dtype=np.float32)
    for i, v in enumerate(m_basis.base):
        for j, w in enumerate(nm_basis.base):
            targ = v+w
            index = basis.rep_to_index(targ)
            if index == None:
                continue
            coef = vect[index]
            if coef != 0:      
                aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
                aux_inv = lambda x: np.prod(np.sqrt([np.math.factorial(o) for o in x]))
                coef = coef * aux(v) * aux(w) * aux_inv(targ)
                #coef = coef 
                #print(v,w,coef)
            mat[i,j] = coef
    return mat

# Devuelve la matriz rho M asociada al vector
def rho_m(basis, m, vect, m_basis = None, nm_basis = None):
    g = gamma(basis, m, vect, m_basis, nm_basis)
    return np.dot(g,np.transpose(g))

# Devuelve la matriz rho 1 asociada al vector
def rho_1(basis, vect):
    d = basis.d
    mat = dok_matrix((d, d), dtype=np.float32)
    for i in range(0, d):
        for j in range(0, d):
            v = bdb(basis,j,i)*vect
            mat[i,j] = np.inner(vect,v)
    return mat
      
# Devuelve la matriz rho 2 asociada al vector
def rho_2(basis, vect, ml_basis = None, mll_basis = None, t_basis = None):
    d = basis.d
    # Creo las bases si no están dadad
    if ml_basis == None or mll_basis == None or t_basis == None:
        ml_basis = fixed_basis(m-1,d)
        mll_basis = fixed_basis(m-2,d)
        t_basis = fixed_basis(2,d)
    mat = dok_matrix((t_basis.size, t_basis.size), dtype=np.float32)
    for i, v in enumerate(t_basis.base):
        for j, w in enumerate(t_basis.base):
            # Creacion de los a
            i_set = t_basis.rep_to_exi(v)
            b_m = np.dot(b(ml_basis, mll_basis, i_set[1]),b(basis, ml_basis, i_set[0]))
            
            # Creacion de los ad
            i_set = t_basis.rep_to_exi(w)
            bd_m = np.dot(bd(ml_basis, basis, i_set[1]),bd(mll_basis, ml_basis, i_set[0]))    

            # Mult de b's y filleo de mat
            coef = np.inner(vect,np.dot(bd_m,b_m)*vect)
            aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
            mat[i,j] = coef * aux(v) * aux(w)
    return mat    

In [24]:
m = 4
d = 4
import time
ti = time.time()
base = fixed_basis(m, d).base
tf = time.time()
print(tf-ti)
print(base)

0.0007023811340332031
[[0 0 0 4]
 [0 0 1 3]
 [0 0 2 2]
 [0 0 3 1]
 [0 0 4 0]
 [0 1 0 3]
 [0 1 1 2]
 [0 1 2 1]
 [0 1 3 0]
 [0 2 0 2]
 [0 2 1 1]
 [0 2 2 0]
 [0 3 0 1]
 [0 3 1 0]
 [0 4 0 0]
 [1 0 0 3]
 [1 0 1 2]
 [1 0 2 1]
 [1 0 3 0]
 [1 1 0 2]
 [1 1 1 1]
 [1 1 2 0]
 [1 2 0 1]
 [1 2 1 0]
 [1 3 0 0]
 [2 0 0 2]
 [2 0 1 1]
 [2 0 2 0]
 [2 1 0 1]
 [2 1 1 0]
 [2 2 0 0]
 [3 0 0 1]
 [3 0 1 0]
 [3 1 0 0]
 [4 0 0 0]]


In [29]:
# Prueba de las matrices gamma (TODO normalizacion)
a = fixed_basis(4, 4)
v = a.rep_to_vect([0, 1, 1, 2])
#+a.rep_to_vect([1, 1, 2, 0])
print(v)
gamma(a, 2, v)
v = np.array([random.random() for a in range(0,a.size)])
v = v / np.linalg.norm(v)
print(v)
gamma(a, 2, v)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 2] [0 1 1 0] 1.0
[0 0 1 1] [0 1 0 1] 1.4142135623730951
[0 1 0 1] [0 0 1 1] 1.4142135623730951
[0 1 1 0] [0 0 0 2] 1.0
[0.00245818 0.12429807 0.09235737 0.05673843 0.05200341 0.23388029
 0.23401698 0.14239911 0.00137916 0.10011861 0.21210931 0.07653313
 0.12088992 0.06257756 0.28254429 0.13310775 0.14610995 0.00906383
 0.06093948 0.15766386 0.15810107 0.10776898 0.11063491 0.14373242
 0.28678535 0.25566518 0.05989708 0.27571683 0.0874394  0.05737582
 0.27334925 0.21140572 0.20371338 0.27734277 0.26057538]
[0 0 0 2] [0 0 0 2] 0.006021284932702713
[0 0 0 2] [0 0 1 1] 0.21529057693064307
[0 0 0 2] [0 0 2 0] 0.09235737145334309
[0 0 0 2] [0 1 0 1] 0.40509254671795236
[0 0 0 2] [0 1 1 0] 0.23401697586066175
[0 0 0 2] [0 2 0 0] 0.10011861433086211
[0 0 0 2] [1 0 0 1] 0.23054938548114984
[0 0 0 2] [1 0 1 0] 0.14610994956373693
[0 0 0 2] [1 1 0 0] 0.15766386006695876
[0 0 0 2] [2 

<10x10 sparse matrix of type '<class 'numpy.float32'>'
	with 100 stored elements in Dictionary Of Keys format>

In [63]:
# Prueba de funcionamiento de las matrices bdb y bbd
v = [1, 1, 0, 2]
vect = a.rep_to_vect(v)
res = bbd(a,0,1) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(a.vect_to_repr(res))
else:
    res = 0
    print(0)


[0 2 0 2]


In [69]:
# Prueba de funcionamiento de las matrices b y bd

l = fixed_basis(3, 3)
m = fixed_basis(2, 3)

vect = l.rep_to_vect([0,1,2])
res = b(l,m,2) * vect
if np.linalg.norm(res) != 0:
    print(res)
    res = res / np.linalg.norm(res)
    print(m.vect_to_repr(res))
    
vect = m.rep_to_vect([0,0,2])
res = bd(m,l,2) * vect
if np.linalg.norm(res) != 0:
    print(res)
    res = res / np.linalg.norm(res)
    print(l.vect_to_repr(res))

[0.         1.41421354 0.         0.         0.         0.        ]
[0 1 1]
[1.73205078 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
[0 0 3]


## Pairing

Construimos el espacio para n particulas en d estados. El hamiltoniano de pairing posee energias equiespaciadas con degeneración doble (asociada a cada estado y su inverso temporal), así como un término de interacción. Por tal motivo, crearemos un espacio con d = 2d.

In [16]:
m = 6
d = 3
# Creo las bases para no tener que recrearlas luego
basis = fixed_basis(m, 2*d)
basis_m1 = fixed_basis(m-1, 2*d)
basis_m2 = fixed_basis(m-2, 2*d)
print(basis.base)
print(basis.size)



[[0 0 0 0 0 6]
 [0 0 0 0 1 5]
 [0 0 0 0 2 4]
 ...
 [5 0 1 0 0 0]
 [5 1 0 0 0 0]
 [6 0 0 0 0 0]]
462


In [17]:
# Parametros hamiltoniano
e = 1
eps = 0.0001
e0 = np.zeros(2*d)
eigenspace_tol = 0.0001
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

def hamiltonian(g, basis, basis_m1, basis_m2):
    # Construccion de H
    h0 = sum([e0[k]*bdb(basis,k,k) for k in range(0,2*d)])
    hi = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k in range(0,d):
        for kb in range(0,d):
            hi += -1*g*bd(basis_m1, basis, 2*k)*bd(basis_m2, basis_m1, 2*k+1)*b(basis_m1, basis_m2, 2*kb+1)*b(basis, basis_m1, 2*kb)

    return h0+hi

def solve(g, m, last_step = None):
    h = hamiltonian(g, basis, basis_m1, basis_m2)
    sol = linalg.eigsh(h,which='SA',k=19)
    if type(last_step) != type(None):
        # Seleccionamos todos los autovects que difieren sus autovalores menos que tol (mismo autoespacio)
        # y tomamos la proyección en el autoespacio de la solución del paso anterior (last_step)
        eig = sol[0].real
        eigv = sol[1]
        cand = [eigv[:,i].real  for (i, x) in enumerate(eig) if abs(x-min(eig)) < eigenspace_tol]
        cand_norm = [x/np.linalg.norm(x) for x in cand]
        fund = np.zeros(len(cand[0]))
        for x in cand_norm:
            fund += np.dot(last_step,x) * x
        #print(np.dot(fund,last_step))
    else:
        argmin = np.argmin(sol[0].real)
        fund = sol[1][:,argmin]
    fund = fund.real / np.linalg.norm(fund)
    return fund

In [7]:
# Rutina de resolución
m = 1
m_basis = fixed_basis(m, 2*d)
nm_basis = fixed_basis(basis.m-m, 2*d)

num = 100
g_range = np.linspace(0.01,5,num)
rho_range= {}

def compute_g(g):
    print(g)
    fund = solve(g, m)
    rho = rho_m(basis, m, fund, m_basis, nm_basis).todense()
    r = np.sort(linalg_d.eigvals(rho).real)
    rho_range[g] = r

Parallel(n_jobs=8, require='sharedmem')(
    delayed(compute_g)(g) for g in g_range);

0.01
0.060404040404040410.11080808080808081

0.16121212121212125
0.21161616161616165
0.26202020202020204
0.3124242424242425
0.3628282828282829
0.4132323232323233
0.4636363636363637
0.5140404040404041
0.5644444444444445
0.614848484848485
0.6652525252525253
0.7156565656565658
0.7660606060606061
0.81646464646464660.8668686868686870.9172727272727274


0.9676767676767678
1.0180808080808081
1.0684848484848486
1.118888888888889
1.1692929292929295
1.21969696969697
1.2701010101010102
1.3205050505050506
1.370909090909091
1.4213131313131315
1.471717171717172
1.5221212121212122
1.5725252525252527
1.622929292929293
1.6733333333333336
1.723737373737374
1.7741414141414142
1.8245454545454547
1.8749494949494951
1.92535353535353561.975757575757576

2.026161616161616
2.0765656565656565
2.126969696969697
2.1773737373737374
2.227777777777778
2.2781818181818183
2.3285858585858588
2.378989898989899
2.4293939393939397
2.4797979797979797
2.53020202020202
2.5806060606060606
2.631010101010101
2.6814141414141415


In [178]:
# Ploteamos
rho_range = dict(sorted(rho_range.items()))
x_axis = list(rho_range.keys())
values = list(rho_range.items())
size = len(values[0][1])

fig = go.Figure()
for k in range(0,size):
    fig.add_trace(go.Scatter(
        x=x_axis,
        y=[values[j][1][k] for j in range(0,num)]
    ))
fig.update_layout(xaxis_title='G',
                   yaxis_title='Rho2')
fig.show()

In [10]:
m = 6
d = 3
# Creo las bases para no tener que recrearlas luego
basis = fixed_basis(m, 2*d)
basis_m1 = fixed_basis(m-1, 2*d)
basis_m2 = fixed_basis(m-2, 2*d)
print(basis.base)
e = 1
eps = 0
e0 = np.zeros(2*d)
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

h = hamiltonian(2, basis, basis_m1, basis_m2)
ha = np.array(h.todense())
ha.astype(np.float32).tofile("myData.dat")
basis.size

[[0 0 0 0 0 6]
 [0 0 0 0 1 5]
 [0 0 0 0 2 4]
 ...
 [5 0 1 0 0 0]
 [5 1 0 0 0 0]
 [6 0 0 0 0 0]]


462

In [7]:
h = hamiltonian(100, basis, basis_m1, basis_m2)
sol = linalg.eigsh(h,which='SA',k=19,return_eigenvectors=True)
eig = sol[0].real
eigv = sol[1]
argmin = np.argmin(sol[0].real)
fund = sol[1][:,argmin]
fund = fund.real / np.linalg.norm(fund)
rho = rho_m(basis, 2, fund).todense()
print(linalg_d.eigvals(rho).real)
rho = rho_2(basis, fund).todense()
print(linalg_d.eigvals(rho).real)
rho_eigv = linalg.eigs(rho)[1].real
rho_eigv

[0.48417228 5.000593   0.5161714  0.48417243 0.49063432 0.5161724
 0.5090997  0.49194258 0.49194294 0.5079397  0.49194244 0.49194297
 0.50794005 0.5079391  0.5079398  0.4999542  0.49981964 0.49981955
 0.49995437 0.49995407 0.49995428]
[0.4841722  5.0005913  0.48417202 0.4906348  0.49194294 0.51617193
 0.5161722  0.4919424  0.491943   0.50909996 0.4998196  0.4919428
 0.49982002 0.49995354 0.4999541  0.499954   0.5079401  0.5079393
 0.50794005 0.50793946 0.49995422]


array([[ 1.01255416e-07, -6.05277023e-07, -2.50204744e-06,
        -1.80095628e-06,  2.16896666e-07, -6.86135024e-07],
       [ 5.68383992e-01,  4.75688876e-06,  2.74343256e-06,
        -2.13515744e-01,  2.07419635e-05,  4.05800211e-06],
       [ 5.49574111e-08, -1.33963147e-06, -1.09084544e-06,
        -8.55059113e-07, -2.19452727e-06,  1.03816944e-06],
       [-9.97826248e-08, -2.76343997e-07,  3.12252268e-07,
         2.75989055e-06, -5.78257050e-06,  2.21217738e-06],
       [ 5.72463641e-08,  1.72084000e-08,  5.80717426e-07,
        -6.36766754e-06, -5.63760125e-07,  7.27958832e-06],
       [-1.53052781e-07, -5.65955602e-08, -8.25953521e-07,
         1.88115591e-05,  4.71133762e-06,  2.70322835e-05],
       [ 2.03850306e-07,  2.57979149e-08, -1.24970356e-07,
        -6.60809792e-06,  3.75639365e-06, -4.71792964e-06],
       [-1.41833416e-08,  7.96038876e-07,  1.99088561e-07,
         2.17394472e-06,  2.60718366e-07, -7.42856764e-06],
       [ 5.77183545e-01,  6.66484220e-06,  5.490

In [18]:
rho1 = rho_1(basis, fund).todense()
print(np.trace(rho1))
print(linalg_d.eigvals(rho1).real)

3.9999998
[1.8763163  1.876315   0.12368434 0.12368418]


In [13]:
rho2 = rho_2(basis, fund).todense()
print(np.trace(rho2))
print(linalg_d.eigvals(rho2).real)
np.set_printoptions(suppress=True)


6.0
[0.00731809 3.7699203  0.00731803 0.01199096 0.88363785 0.10904462
 0.88363725 0.10904456 0.10904457 0.10904454]


In [28]:
g = gamma(basis, 2, fund).todense()
rho = np.dot(g,np.transpose(g))
print(np.trace(rho))
fund

0 2
[0 0 0 2] [0 0 2 0] 0.08554583787918091
1 1
[0 0 1 1] [0 0 1 1] 0.17109167575836184
1 8
[0 0 1 1] [1 1 0 0] 0.3302189111709595
2 0
[0 0 2 0] [0 0 0 2] 0.08554583787918091
3 7
[0 1 0 1] [1 0 1 0] 0.3302189111709595
4 6
[0 1 1 0] [1 0 0 1] 0.3302189111709595
5 9
[0 2 0 0] [2 0 0 0] 0.9400199055671692
6 4
[1 0 0 1] [0 1 1 0] 0.3302189111709595
7 3
[1 0 1 0] [0 1 0 1] 0.3302189111709595
8 1
[1 1 0 0] [0 0 1 1] 0.3302189111709595
8 8
[1 1 0 0] [1 1 0 0] 1.8800398111343388
9 5
[2 0 0 0] [0 2 0 0] 0.9400199055671692
6.0000005


array([ 0.00000019, -0.00000003,  0.08554584, -0.00000006,  0.00000006,
        0.00000007, -0.00000002, -0.00000005,  0.00000001, -0.        ,
       -0.00000008,  0.00000001,  0.00000005,  0.00000005, -0.        ,
        0.00000008,  0.00000007, -0.00000007,  0.00000002, -0.00000004,
        0.3302189 ,  0.00000004, -0.00000001, -0.00000015, -0.00000048,
        0.00000002,  0.00000011, -0.00000001,  0.00000016, -0.00000014,
        0.9400199 , -0.00000003,  0.00000011,  0.0000002 , -0.00000003],
      dtype=float32)

In [6]:
m = 6
d = 3
basis = fixed_basis(m, 2*d)
m = 2
m_basis = fixed_basis(m, basis.d)
nm_basis = fixed_basis(basis.m-m, basis.d)

h = hamiltonian(1, basis, basis_m1, basis_m2)
sol = linalg.eigsh(h,which='SA',k=19,return_eigenvectors=True)
eig = sol[0].real
eigv = sol[1]
argmin = np.argmin(sol[0].real)
fund = sol[1][:,argmin]
fund = fund.real / np.linalg.norm(fund)
g = gamma(basis, 2, fund).todense()

U, s, Vh = np.linalg.svd(g, full_matrices=False)

parsum = []
for nu in range(0,s.size):
    if nu == 0:
        v = np.zeros(basis.size)
    else:
        v = parsum[nu-1]
    for i, vi in enumerate(m_basis.base):
        for j, wi in enumerate(nm_basis.base):
            targ = vi+wi
            index = basis.rep_to_index(targ)
            aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
            aux_inv = lambda x: np.prod(np.sqrt([np.math.factorial(o) for o in x]))
            v += 1/binom(basis.m,m) * np.sqrt(s[nu]) * U[i,nu] * Vh[nu,j] * basis.canonicals[index]
    parsum.append(v)

[0 0 0 0 0 2] [0 0 0 0 0 4] -1.6619690476823453e-08
[0 0 0 0 0 2] [0 0 0 0 1 3] -2.3768749838946623e-08
[0 0 0 0 0 2] [0 0 0 0 2 2] 1.9525295758141683e-08
[0 0 0 0 0 2] [0 0 0 0 3 1] 0.058295420181347896
[0 0 0 0 0 2] [0 0 0 0 4 0] 4.153578370136302e-09
[0 0 0 0 0 2] [0 0 0 1 0 3] 2.441536391252198e-08
[0 0 0 0 0 2] [0 0 0 1 1 2] -4.419602155464948e-08
[0 0 0 0 0 2] [0 0 0 1 2 1] 1.2241738913643124e-08
[0 0 0 0 0 2] [0 0 0 1 3 0] 5.146590265070472e-09
[0 0 0 0 0 2] [0 0 0 2 0 2] 6.020629597768754e-08
[0 0 0 0 0 2] [0 0 0 2 1 1] 3.131564118239512e-08
[0 0 0 0 0 2] [0 0 0 2 2 0] 8.935250761510359e-09
[0 0 0 0 0 2] [0 0 0 3 0 1] -9.139666658776618e-10
[0 0 0 0 0 2] [0 0 0 3 1 0] -1.5666566710592633e-08
[0 0 0 0 0 2] [0 0 0 4 0 0] 4.424912880551801e-09
[0 0 0 0 0 2] [0 0 1 0 0 3] 5.5164501713031866e-08
[0 0 0 0 0 2] [0 0 1 0 1 2] -2.4610945101311765e-08
[0 0 0 0 0 2] [0 0 1 0 2 1] -1.0048680734255497e-08
[0 0 0 0 0 2] [0 0 1 0 3 0] 1.6680134606872343e-08
[0 0 0 0 0 2] [0 0 1 1 0 2] 4.97118

[0 0 0 0 2 0] [1 1 2 0 0 0] 3.4605140974974806e-09
[0 0 0 0 2 0] [1 2 0 0 0 1] 9.955468449618365e-09
[0 0 0 0 2 0] [1 2 0 0 1 0] 2.9604607444492814e-08
[0 0 0 0 2 0] [1 2 0 1 0 0] -4.219963822826856e-10
[0 0 0 0 2 0] [1 2 1 0 0 0] 4.353335025797378e-09
[0 0 0 0 2 0] [1 3 0 0 0 0] -3.9248098104849305e-09
[0 0 0 0 2 0] [2 0 0 0 0 2] -1.3405959897738738e-08
[0 0 0 0 2 0] [2 0 0 0 1 1] -4.8417719821272695e-09
[0 0 0 0 2 0] [2 0 0 0 2 0] -1.2782201730225727e-08
[0 0 0 0 2 0] [2 0 0 1 0 1] 2.265729825978724e-09
[0 0 0 0 2 0] [2 0 0 1 1 0] -7.447044519411359e-09
[0 0 0 0 2 0] [2 0 0 2 0 0] 2.8721376388318727e-09
[0 0 0 0 2 0] [2 0 1 0 0 1] -4.059112601595416e-09
[0 0 0 0 2 0] [2 0 1 0 1 0] 1.1316033906638971e-08
[0 0 0 0 2 0] [2 0 1 1 0 0] -2.4262476472358685e-09
[0 0 0 0 2 0] [2 0 2 0 0 0] 1.4863094088823914e-09
[0 0 0 0 2 0] [2 1 0 0 0 1] 2.91176984745789e-08
[0 0 0 0 2 0] [2 1 0 0 1 0] 1.4437303867188718e-08
[0 0 0 0 2 0] [2 1 0 1 0 0] -2.6836075583958063e-09
[0 0 0 0 2 0] [2 1 1 0 0 0] -7

[0 0 0 2 0 0] [0 4 0 0 0 0] -2.167132917563208e-09
[0 0 0 2 0 0] [1 0 0 0 0 3] 6.521953116456203e-11
[0 0 0 2 0 0] [1 0 0 0 1 2] -7.488097253371961e-09
[0 0 0 2 0 0] [1 0 0 0 2 1] -1.483087253006943e-08
[0 0 0 2 0 0] [1 0 0 0 3 0] -6.771928795501481e-09
[0 0 0 2 0 0] [1 0 0 1 0 2] -4.5328218742672154e-09
[0 0 0 2 0 0] [1 0 0 1 1 1] 5.818219743907175e-09
[0 0 0 2 0 0] [1 0 0 1 2 0] 2.1691011410555853e-08
[0 0 0 2 0 0] [1 0 0 2 0 1] 7.520109810948695e-10
[0 0 0 2 0 0] [1 0 0 2 1 0] -9.208931311011957e-09
[0 0 0 2 0 0] [1 0 0 3 0 0] 4.743044032210247e-09
[0 0 0 2 0 0] [1 0 1 0 0 2] -3.0988247523566774e-09
[0 0 0 2 0 0] [1 0 1 0 1 1] 5.751908727091859e-09
[0 0 0 2 0 0] [1 0 1 0 2 0] 2.640466734149527e-09
[0 0 0 2 0 0] [1 0 1 1 0 1] 1.02474020058216e-08
[0 0 0 2 0 0] [1 0 1 1 1 0] 1.2820802205763954e-08
[0 0 0 2 0 0] [1 0 1 2 0 0] -2.766895131061652e-09
[0 0 0 2 0 0] [1 0 2 0 0 1] 1.694541040819786e-08
[0 0 0 2 0 0] [1 0 2 0 1 0] 1.7262928864170135e-08
[0 0 0 2 0 0] [1 0 2 1 0 0] 3.03539844

[0 0 1 1 0 0] [0 0 1 0 0 3] -3.0259475022216528e-09
[0 0 1 1 0 0] [0 0 1 0 1 2] -1.0893839078005886e-08
[0 0 1 1 0 0] [0 0 1 0 2 1] -2.0676145644737213e-08
[0 0 1 1 0 0] [0 0 1 0 3 0] -9.92931620774884e-09
[0 0 1 1 0 0] [0 0 1 1 0 2] 1.2071265764745931e-08
[0 0 1 1 0 0] [0 0 1 1 1 1] 0.13832351565361026
[0 0 1 1 0 0] [0 0 1 1 2 0] -1.2984395780790693e-08
[0 0 1 1 0 0] [0 0 1 2 0 1] -2.90705853980523e-08
[0 0 1 1 0 0] [0 0 1 2 1 0] -2.7904800925379733e-08
[0 0 1 1 0 0] [0 0 1 3 0 0] 2.998368388558984e-08
[0 0 1 1 0 0] [0 0 2 0 0 2] 1.5430780232741626e-08
[0 0 1 1 0 0] [0 0 2 0 1 1] 4.324213266997282e-08
[0 0 1 1 0 0] [0 0 2 0 2 0] -9.58075699355689e-09
[0 0 1 1 0 0] [0 0 2 1 0 1] 1.3779926695565056e-08
[0 0 1 1 0 0] [0 0 2 1 1 0] -3.8915818443061035e-08
[0 0 1 1 0 0] [0 0 2 2 0 0] 0.2922311648726462
[0 0 1 1 0 0] [0 0 3 0 0 1] 6.906361038616637e-10
[0 0 1 1 0 0] [0 0 3 0 1 0] -8.14624989686763e-09
[0 0 1 1 0 0] [0 0 3 1 0 0] 9.521041674317586e-08
[0 0 1 1 0 0] [0 0 4 0 0 0] -7.039143667

[0 1 0 0 0 1] [1 2 0 0 1 0] 4.929583827061752e-07
[0 1 0 0 0 1] [1 2 0 1 0 0] 3.4043604507958538e-09
[0 1 0 0 0 1] [1 2 1 0 0 0] 4.3235498833297336e-10
[0 1 0 0 0 1] [1 3 0 0 0 0] 9.541301082549582e-09
[0 1 0 0 0 1] [2 0 0 0 0 2] 1.1740855592499901e-08
[0 1 0 0 0 1] [2 0 0 0 1 1] 4.0186853107834366e-08
[0 1 0 0 0 1] [2 0 0 0 2 0] 2.91176984745789e-08
[0 1 0 0 0 1] [2 0 0 1 0 1] 2.3693045332156454e-09
[0 1 0 0 0 1] [2 0 0 1 1 0] 2.1907251124275717e-08
[0 1 0 0 0 1] [2 0 0 2 0 0] 1.7922060502328918e-09
[0 1 0 0 0 1] [2 0 1 0 0 1] -8.803670449381663e-09
[0 1 0 0 0 1] [2 0 1 0 1 0] 7.515395594737129e-08
[0 1 0 0 0 1] [2 0 1 1 0 0] 3.290067240868666e-08
[0 1 0 0 0 1] [2 0 2 0 0 0] 6.579911060455856e-10
[0 1 0 0 0 1] [2 1 0 0 0 1] 2.7482959907842996e-08
[0 1 0 0 0 1] [2 1 0 0 1 0] 0.46021745469288616
[0 1 0 0 0 1] [2 1 0 1 0 0] -1.3607037979271368e-08
[0 1 0 0 0 1] [2 1 1 0 0 0] 1.3857072063131842e-08
[0 1 0 0 0 1] [2 2 0 0 0 0] -1.9424686045261785e-07
[0 1 0 0 0 1] [3 0 0 0 0 1] 4.712938996

[0 1 1 0 0 0] [0 2 0 0 2 0] 2.3874173062478e-10
[0 1 1 0 0 0] [0 2 0 1 0 1] 7.003666628219364e-10
[0 1 1 0 0 0] [0 2 0 1 1 0] 7.073107502972435e-09
[0 1 1 0 0 0] [0 2 0 2 0 0] 3.649353204299328e-09
[0 1 1 0 0 0] [0 2 1 0 0 1] 7.056112692426269e-09
[0 1 1 0 0 0] [0 2 1 0 1 0] -1.2688904260970339e-09
[0 1 1 0 0 0] [0 2 1 1 0 0] 3.297243425803528e-08
[0 1 1 0 0 0] [0 2 2 0 0 0] 1.746233957433673e-08
[0 1 1 0 0 0] [0 3 0 0 0 1] -6.6086287553446255e-09
[0 1 1 0 0 0] [0 3 0 0 1 0] -1.0553738327701012e-08
[0 1 1 0 0 0] [0 3 0 1 0 0] 4.1604494072089444e-08
[0 1 1 0 0 0] [0 3 1 0 0 0] 9.807615201091341e-09
[0 1 1 0 0 0] [0 4 0 0 0 0] -1.7749859026138743e-08
[0 1 1 0 0 0] [1 0 0 0 0 3] 1.9967297149037225e-08
[0 1 1 0 0 0] [1 0 0 0 1 2] 1.9558876740433107e-10
[0 1 1 0 0 0] [1 0 0 0 2 1] -6.281317688205945e-09
[0 1 1 0 0 0] [1 0 0 0 3 0] -1.4854447449863528e-08
[0 1 1 0 0 0] [1 0 0 1 0 2] 8.467004875001294e-09
[0 1 1 0 0 0] [1 0 0 1 1 1] 0.1627281904220581
[0 1 1 0 0 0] [1 0 0 1 2 0] -1.0103682335

[1 0 0 0 0 1] [1 1 1 0 1 0] 1.0628374376676261e-07
[1 0 0 0 0 1] [1 1 1 1 0 0] 4.652857713155896e-08
[1 0 0 0 0 1] [1 1 2 0 0 0] 9.305399460905408e-10
[1 0 0 0 0 1] [1 2 0 0 0 1] 2.7482959907842996e-08
[1 0 0 0 0 1] [1 2 0 0 1 0] 0.46021745469288616
[1 0 0 0 0 1] [1 2 0 1 0 0] -1.3607037979271368e-08
[1 0 0 0 0 1] [1 2 1 0 0 0] 1.3857072063131842e-08
[1 0 0 0 0 1] [1 3 0 0 0 0] -1.5860189741550765e-07
[1 0 0 0 0 1] [2 0 0 0 0 2] 2.0013978807575934e-09
[1 0 0 0 0 1] [2 0 0 0 1 1] -3.058839186001503e-09
[1 0 0 0 0 1] [2 0 0 0 2 0] 2.661245173677999e-09
[1 0 0 0 0 1] [2 0 0 1 0 1] 3.455801192704259e-10
[1 0 0 0 0 1] [2 0 0 1 1 0] 1.0713122223577878e-09
[1 0 0 0 0 1] [2 0 0 2 0 0] -3.90889086297007e-09
[1 0 0 0 0 1] [2 0 1 0 0 1] 4.33566920168576e-10
[1 0 0 0 0 1] [2 0 1 0 1 0] -8.12234901724539e-09
[1 0 0 0 0 1] [2 0 1 1 0 0] 9.45952496477145e-09
[1 0 0 0 0 1] [2 0 2 0 0 0] 1.8095630769160324e-09
[1 0 0 0 0 1] [2 1 0 0 0 1] 8.163049795408272e-09
[1 0 0 0 0 1] [2 1 0 0 1 0] -7.285328434107

[1 0 1 0 0 0] [0 0 0 0 1 3] -1.557632423043742e-08
[1 0 1 0 0 0] [0 0 0 0 2 2] 3.7805179431416036e-08
[1 0 1 0 0 0] [0 0 0 0 3 1] 5.299257921365098e-09
[1 0 1 0 0 0] [0 0 0 0 4 0] 1.0473623746065641e-08
[1 0 1 0 0 0] [0 0 0 1 0 3] -4.993956359555796e-09
[1 0 1 0 0 0] [0 0 0 1 1 2] 7.842458238371819e-09
[1 0 1 0 0 0] [0 0 0 1 2 1] 4.274725462494189e-09
[1 0 1 0 0 0] [0 0 0 1 3 0] 3.1646614218061586e-09
[1 0 1 0 0 0] [0 0 0 2 0 2] -3.098824752356677e-09
[1 0 1 0 0 0] [0 0 0 2 1 1] 5.751908727091859e-09
[1 0 1 0 0 0] [0 0 0 2 2 0] 2.640466734149527e-09
[1 0 1 0 0 0] [0 0 0 3 0 1] 5.916340306555413e-09
[1 0 1 0 0 0] [0 0 0 3 1 0] 7.402093604724769e-09
[1 0 1 0 0 0] [0 0 0 4 0 0] -1.129580207148706e-09
[1 0 1 0 0 0] [0 0 1 0 0 3] 8.409436561967737e-09
[1 0 1 0 0 0] [0 0 1 0 1 2] 1.700912311126514e-08
[1 0 1 0 0 0] [0 0 1 0 2 1] -7.795618335800069e-09
[1 0 1 0 0 0] [0 0 1 0 3 0] -8.913749003623e-09
[1 0 1 0 0 0] [0 0 1 1 0 2] -1.0990783023784102e-09
[1 0 1 0 0 0] [0 0 1 1 1 1] 4.470813405837

[2 0 0 0 0 0] [0 0 3 1 0 0] -8.053114619599455e-09
[2 0 0 0 0 0] [0 0 4 0 0 0] -3.2753151302244983e-09
[2 0 0 0 0 0] [0 1 0 0 0 3] 6.778586136846343e-09
[2 0 0 0 0 0] [0 1 0 0 1 2] 2.841639634709736e-08
[2 0 0 0 0 0] [0 1 0 0 2 1] 2.91176984745789e-08
[2 0 0 0 0 0] [0 1 0 0 3 0] 8.335381274093834e-09
[2 0 0 0 0 0] [0 1 0 1 0 2] 1.6753513021328104e-09
[2 0 0 0 0 0] [0 1 0 1 1 1] 2.1907251124275717e-08
[2 0 0 0 0 0] [0 1 0 1 2 0] -2.6836075583958063e-09
[2 0 0 0 0 0] [0 1 0 2 0 1] 1.792206050232892e-09
[2 0 0 0 0 0] [0 1 0 2 1 0] 1.8269984414232e-09
[2 0 0 0 0 0] [0 1 0 3 0 0] -5.819703607912174e-10
[2 0 0 0 0 0] [0 1 1 0 0 2] -6.225135074089394e-09
[2 0 0 0 0 0] [0 1 1 0 1 1] 7.515395594737129e-08
[2 0 0 0 0 0] [0 1 1 0 2 0] -7.912640320739683e-09
[2 0 0 0 0 0] [0 1 1 1 0 1] 3.290067240868666e-08
[2 0 0 0 0 0] [0 1 1 1 1 0] 3.601818576726146e-08
[2 0 0 0 0 0] [0 1 1 2 0 0] 4.5471601595181716e-08
[2 0 0 0 0 0] [0 1 2 0 0 1] 6.579911060455856e-10
[2 0 0 0 0 0] [0 1 2 0 1 0] 1.433163365760

In [13]:
np.linalg.norm(np.dot(np.dot(U,np.diag(s)),Vh)-g)

2.473555e-07

In [12]:
np.linalg.norm(g)

3.8729837

In [7]:
v = np.zeros(basis.size)
for i, vi in enumerate(m_basis.base):
    for j, wi in enumerate(nm_basis.base):
        targ = vi+wi
        index = basis.rep_to_index(targ)
        #aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
        #aux_inv = lambda x: np.prod(np.sqrt([np.math.factorial(o) for o in x]))
        v += 1/binom(basis.m,m) * g[i,j] * basis.canonicals[index] 

In [9]:
np.linalg.norm(v/np.linalg.norm(v)-fund)

0.2714044096214872

In [15]:
basis = fixed_basis(4, 4)
m = 2
m_basis = fixed_basis(m, basis.d)
nm_basis = fixed_basis(basis.m-m, basis.d)

v = basis.rep_to_vect([0, 1, 1, 2])+basis.rep_to_vect([1, 1, 2, 0])+basis.rep_to_vect([4,0,0,0])
print(v)
g = gamma(basis, 2, v)
v = np.zeros(basis.size)
for i, vi in enumerate(m_basis.base):
    for j, wi in enumerate(nm_basis.base):
        targ = vi+wi
        index = basis.rep_to_index(targ)
        aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
        aux_inv = lambda x: np.prod(np.sqrt([np.math.factorial(o) for o in x]))
        v += 1/binom(basis.m,m) * g[i,j] * basis.canonicals[index] * aux(wi) * aux(vi) * aux_inv(targ)

print(v)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0 0 0 2] [0 1 1 0] 1.0
[0 0 1 1] [0 1 0 1] 1.4142135623730951
[0 0 2 0] [1 1 0 0] 1.0
[0 1 0 1] [0 0 1 1] 1.4142135623730951
[0 1 1 0] [0 0 0 2] 1.0
[0 1 1 0] [1 0 1 0] 1.4142135623730951
[1 0 1 0] [0 1 1 0] 1.4142135623730951
[1 1 0 0] [0 0 2 0] 1.0
[2 0 0 0] [2 0 0 0] 2.4494897427831774
[0.         0.         0.         0.         0.         0.
 0.99999999 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.99999999 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         1.00000004]


In [66]:
nu=0
w = np.zeros(basis.size)
for i, vi in enumerate(m_basis.base):
    for j, wi in enumerate(nm_basis.base):
        targ = vi+wi
        index = basis.rep_to_index(targ)
        w += 1/binom(basis.m,m) * np.sqrt(s[nu]) * U[i,nu] * Vh[nu,j] * basis.canonicals[index]


In [67]:
np.dot(fund,w)

0.12158063129027116

In [40]:
binom(basis.m,m)

15.0